In [1]:
# Importing all the dependencies
import pandas as pd
import os
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, NamedStyle, Border, Side, PatternFill

In [2]:
######Configure inputs here
# Reading the required files

##########Configuration of filenames
cwd = os.getcwd()
#input_demand = "28-04-2024_Demand_Automation.xlsx"
standard_costs = os.path.join(cwd, "client_data", "Data 3 entities.xlsx")

input_routing = os.path.join(cwd, "client_data", "routing data - 070424.xlsx")
routing_additional = os.path.join(cwd, "client_data", "Routing - Factory.xlsx")

# Read the Capacity_DD_Filtered file
df_production = pd.read_excel(r"Capacity_DD_Filtered.xlsx")
# Remove rows where Throughput is zero
df_production = df_production[df_production['Throughput'] > 0]
# Reset the index after filtering
df_production.reset_index(drop=True, inplace=True)


df_3=pd.read_excel(standard_costs,skiprows=2)

input_demand = "28-04-2024_Demand_Automation.xlsx"

In [3]:
#######Adding missing routing file here so that it can be included in preparation
df_missing_routing=pd.read_excel(routing_additional,sheet_name='II. FGs_missing_routing.',skiprows=2)

df_missing_routing=df_missing_routing[['Factory', 'Item Code', 'Crew\nTotal ', 'Run Rate (Primary UOM)\nFG', 'Power Usage\nFG']].rename(columns={
    'Crew\nTotal ': 'Crew Total',
    'Run Rate (Primary UOM)\nFG': 'Run Rate (Primary UOM FG)',
    'Power Usage\nFG': 'Power Usage FG'
})

df_missing_routing=df_missing_routing.rename(columns={
    'Factory':'Unit'})

conditions = [
    df_missing_routing['Unit'].isin(['NF1', 'NF2']),
    df_missing_routing['Unit'] == 'GFM',
    df_missing_routing['Unit'] == 'KPF'
]
choices = ['NFC', 'GFC', 'KFC']
df_missing_routing['FACTCode'] = np.select(conditions, choices, default=None)
df_missing_routing.to_excel('Missing_routing.xlsx',index=False)

df_routing_missing= pd.read_excel('Missing_routing.xlsx')
##This file will be used later for concatenation

In [4]:
# df_demand=pd.read_excel(r"01-04-2024_Demand_Automation.xlsx",sheet_name='Demand_output')
df_demand=pd.read_excel(input_demand,sheet_name='Demand_output')

df_routing_NFC1=pd.read_excel(input_routing,skiprows=4,sheet_name='NF1')
df_routing_NFC2=pd.read_excel(input_routing,skiprows=4,sheet_name='NF2')
df_routing_KFC=pd.read_excel(input_routing,skiprows=4,sheet_name='KPF')
df_routing_GFC=pd.read_excel(input_routing,skiprows=4,sheet_name='GFM')

df_routing_GFC.rename(columns={
    'Crew\nTotal \nRevised':'Crew\nTotal '
},inplace=True)

# Merging all datasets into one combined DataFrame.
df_routing=pd.concat([df_routing_NFC1, df_routing_NFC2, df_routing_GFC, df_routing_KFC], ignore_index=True)

df_routing = df_routing[['Unit', 'Item Code', 'Crew\nTotal ', 'Run Rate (Primary UOM)\nFG', 'Power Usage\nTotal ']].rename(columns={
    'Crew\nTotal ': 'Crew Total',
    'Run Rate (Primary UOM)\nFG': 'Run Rate (Primary UOM FG)',
    'Power Usage\nTotal ': 'Power Usage FG'
})

In [5]:
df_production

,FGSKUCode,Description,Line,Throughput,Factory,Weight UOM,retain_flag
0,4001351001,Beef Hot Dog 450G X 20,F,172.222222,NFC,9.000,1
1,4001351001,Beef Hot Dog 450G X 20,Tiromat,66.000000,KFC,9.000,1
2,4001351022,CRAVES ANGUS BEEF HOTDOG 450GM (20*5*90GM),F,172.222222,NFC,9.000,1
3,4001370158,ZINGZ BREADED CHICKEN BURGER 12 PCS (18*12*56GM),C1,109.375000,NFC,12.096,1
4,4001370158,ZINGZ BREADED CHICKEN BURGER 12 PCS (18*12*56GM),C2,91.145833,NFC,12.096,1
...,...,...,...,...,...,...,...
2513,4001972283,AMC TEMPURA NUGGETS 1000G PP+ POMME FRITES 100...,Rivo,100.000000,KFC,10.000,1
2514,4001972283,AMC TEMPURA NUGGETS 1000G PP+ POMME FRITES 100...,Ultra Formax,100.000000,KFC,10.000,1
2515,4001972283,AMC TEMPURA NUGGETS 1000G PP+ POMME FRITES 100...,Glazed,100.000000,KFC,10.000,1
2516,4001972283,AMC TEMPURA NUGGETS 1000G PP+ POMME FRITES 100...,Tiromat,100.000000,KFC,10.000,1


# Define Exchange Rates

In [6]:
# default_exchange_rates = {'KFC': 3.26796317659, 'NFC': 0.26666666666, 'GFC': 0.2722940776}
## Applying updated Exchange rates for year 2023
default_exchange_rates = {'KFC': 3.25520833333, 'NFC': 0.26666666666, 'GFC': 0.2722940776}

In [7]:
df_routing.columns

Index(['Unit', 'Item Code', 'Crew Total', 'Run Rate (Primary UOM FG)',
       'Power Usage FG'],
      dtype='object')

# Preprocessing Steps

In [8]:
df_routing.head()

,Unit,Item Code,Crew Total,Run Rate (Primary UOM FG),Power Usage FG
0,NF1,4001100156,15.0,147.0,181.0
1,NF1,4001100218,16.0,135.0,181.0
2,NF1,4001310103,13.0,134.0,186.5
3,NF1,4001310105,14.0,123.0,181.0
4,NF1,4001310110,15.0,201.0,181.0


In [9]:
df_routing_missing.head()

,Unit,Item Code,Crew Total,Run Rate (Primary UOM FG),Power Usage FG,FACTCode
0,NF2,4001972056,26,70.0,9.0,NFC
1,GFM,4001971946,36,240.0,9.0,GFC
2,GFM,4001971947,36,240.0,9.0,GFC
3,GFM,4001971948,28,125.0,9.0,GFC
4,NF2,4001380891,34,134.0,16.0,NFC


In [10]:
# Preprocessing Steps for Production File
# Filter the DataFrame based on the 'retain_flag' column
df_production = df_production[df_production['retain_flag'] == 1]

# Select specific columns
selected_columns = ['FGSKUCode', 'Description', 'Factory', 'Line', 'Throughput', 'retain_flag']
df_production = df_production[selected_columns]

In [11]:
df_production.head()

,FGSKUCode,Description,Factory,Line,Throughput,retain_flag
0,4001351001,Beef Hot Dog 450G X 20,NFC,F,172.222222,1
1,4001351001,Beef Hot Dog 450G X 20,KFC,Tiromat,66.000000,1
2,4001351022,CRAVES ANGUS BEEF HOTDOG 450GM (20*5*90GM),NFC,F,172.222222,1
3,4001370158,ZINGZ BREADED CHICKEN BURGER 12 PCS (18*12*56GM),NFC,C1,109.375000,1
4,4001370158,ZINGZ BREADED CHICKEN BURGER 12 PCS (18*12*56GM),NFC,C2,91.145833,1


In [12]:
df_routing.head()

,Unit,Item Code,Crew Total,Run Rate (Primary UOM FG),Power Usage FG
0,NF1,4001100156,15.0,147.0,181.0
1,NF1,4001100218,16.0,135.0,181.0
2,NF1,4001310103,13.0,134.0,186.5
3,NF1,4001310105,14.0,123.0,181.0
4,NF1,4001310110,15.0,201.0,181.0


In [13]:
# Create the FACTCode column based on conditions and choices
conditions = [
    df_routing['Unit'].isin(['NF1', 'NF2']),
    df_routing['Unit'] == 'GFM',
    df_routing['Unit'] == 'KPF'
]
choices = ['NFC', 'GFC', 'KFC']
df_routing['FACTCode'] = np.select(conditions, choices, default=None)
######################## Add missing data#############################

df_routing=pd.concat([df_routing,df_routing_missing])
############################################################

# Preprocessing Steps for 3 Entries

# Rename columns
df_3.rename(columns={
    'Unnamed: 2': 'Type',
    'Unnamed: 3': 'Unit',
    'NFC': 'NFC(SAR)',
    'KFC': 'KFC(KD)',
    'GFC': 'GFC(AED)'
}, inplace=True)

# Filter data for 'Electricity' type
df_Electricity = df_3[df_3['Type'] == 'Electricity ']

# Melt DataFrame
df_Electricity = df_Electricity.melt(id_vars=['Type', 'Unit'], 
                                     value_vars=['NFC(SAR)', 'KFC(KD)', 'GFC(AED)'],
                                     var_name='Factory', 
                                     value_name='Cost')

# Split 'Factory' column
df_Electricity[['FACTCode', 'Currency']] = df_Electricity['Factory'].str.split('(', expand=True)

# Remove ')' from 'Currency' column
df_Electricity['Currency'] = df_Electricity['Currency'].str.rstrip(')')

# Select specific columns
df_Electricity = df_Electricity[['Type', 'FACTCode', 'Currency', 'Cost']]

# Calculate 'Cost (in USD)'
df_Electricity['Cost (in USD)'] = df_Electricity.apply(lambda row: row['Cost'] * default_exchange_rates.get(row['FACTCode'], 1), axis=1)

# Merge the Production and Routing files using the Fact-FG level

In [14]:
#df=pd.merge(df_production,df_routing,left_on=['Factory','FGSKUCode'],right_on=['FACTCode','Item Code'],how='left')
#df.head()
#[df['FGSKUCode']==4001971947]
df = pd.merge(df_production, df_routing, left_on=['Factory', 'FGSKUCode'], right_on=['FACTCode', 'Item Code'], how='left')
df = pd.merge(df, df_Electricity[['FACTCode', 'Cost (in USD)']], on='FACTCode', how='left')
df.head()

,FGSKUCode,Description,Factory,Line,Throughput,retain_flag,Unit,Item Code,Crew Total,Run Rate (Primary UOM FG),Power Usage FG,FACTCode,Cost (in USD)
0,4001351001,Beef Hot Dog 450G X 20,NFC,F,172.222222,1,NF1,4001351001,15.000000,50.000000,482.500000,NFC,0.048000
1,4001351001,Beef Hot Dog 450G X 20,KFC,Tiromat,66.000000,1,KPF,4001351001,20.000001,66.000001,237.960001,KFC,0.009766
2,4001351022,CRAVES ANGUS BEEF HOTDOG 450GM (20*5*90GM),NFC,F,172.222222,1,NF1,4001351022,15.000000,50.000000,482.500000,NFC,0.048000
3,4001370158,ZINGZ BREADED CHICKEN BURGER 12 PCS (18*12*56GM),NFC,C1,109.375000,1,NF2,4001370158,18.000000,140.000000,145.000000,NFC,0.048000
4,4001370158,ZINGZ BREADED CHICKEN BURGER 12 PCS (18*12*56GM),NFC,C2,91.145833,1,NF2,4001370158,18.000000,140.000000,145.000000,NFC,0.048000


In [15]:
# Check duplicate 
def remove_duplicates_keep_first_occurrence(df):
    """
    Remove duplicates in the DataFrame based on 'FactCode', 'FGSKUCode', and 'Line',
    and keep only the first occurrence.
    
    Parameters:
    - df: Input DataFrame
    
    Returns:
    - DataFrame without duplicates based on specified columns
    """
    df_filtered = df.drop_duplicates(subset=['Factory', 'FGSKUCode', 'Line'], keep='first')
    
    return df_filtered

df=remove_duplicates_keep_first_occurrence(df)

In [16]:
df.head(1)

,FGSKUCode,Description,Factory,Line,Throughput,retain_flag,Unit,Item Code,Crew Total,Run Rate (Primary UOM FG),Power Usage FG,FACTCode,Cost (in USD)
0,4001351001,Beef Hot Dog 450G X 20,NFC,F,172.222222,1,NF1,4001351001,15.0,50.0,482.5,NFC,0.048


In [17]:
df=df[['Factory','FGSKUCode','Description','Line','Throughput','Crew Total','Run Rate (Primary UOM FG)','Power Usage FG','Cost (in USD)']]

df['EnergyCostPerUnit'] = (df['Power Usage FG'] * df['Cost (in USD)']) / df['Throughput']
df['MOQ'] = None

df = df[['Factory', 'Line', 'FGSKUCode', 'Throughput', 'EnergyCostPerUnit', 'Crew Total']]
df = df.rename(columns={
    'Factory': 'FactCode',
    'Line': 'LineNum',
    'FGSKUCode': 'Unified code',
    'Throughput': 'ThruputPerHour',
    'Crew Total': 'CrewSize'
})

# Add MOQ column (as it was in the original code)
df['MOQ'] = None

# Reorder columns to match the original output
df = df[['FactCode', 'LineNum', 'Unified code', 'MOQ', 'EnergyCostPerUnit', 'CrewSize', 'ThruputPerHour']]
### Adding Retain Flag Logic
df['retain_flag'] = df.apply(lambda row: 0 if pd.isnull(row['EnergyCostPerUnit']) or pd.isnull(row['CrewSize'])or pd.isnull(row['ThruputPerHour']) else 1, axis=1)



In [18]:
df.to_csv('df_final.csv')

# QC


In [19]:
df_final = df.copy()
#For QC Purpose
df_prod_uniquefg = df_final[['Unified code']].drop_duplicates()

# Print the unique DataFrame

df_merge= pd.merge(df_demand,df_prod_uniquefg, left_on='FGSKUCode',right_on='Unified code' ,how='inner')
df_merge

,FGSKUCode,Country,Month,UM,Demand,Ret_Flag,Unified code
0,4001351001,IS & PL_Cote d'Ivoire,1,CARTON,0.000000,0,4001351001
1,4001351001,IS & PL_Cote d'Ivoire,2,CARTON,0.000000,0,4001351001
2,4001351001,IS & PL_Cote d'Ivoire,3,CARTON,0.000000,0,4001351001
3,4001351001,IS & PL_Cote d'Ivoire,4,CARTON,0.000000,0,4001351001
4,4001351001,IS & PL_Cote d'Ivoire,5,CARTON,0.000000,0,4001351001
...,...,...,...,...,...,...,...
14803,4001972283,UAE & LG_UAE,8,CARTON,1057.728780,1,4001972283
14804,4001972283,UAE & LG_UAE,9,CARTON,0.000000,0,4001972283
14805,4001972283,UAE & LG_UAE,10,CARTON,2134.846639,1,4001972283
14806,4001972283,UAE & LG_UAE,11,CARTON,787.606649,1,4001972283


In [20]:
# Get the number of distinct values for each column
distinct_counts = df_final.nunique()

# Get the number of null values for each column
null_counts = df_final.isnull().sum()
counts=df_final.count()
# Get the count, mean, std, min, and percentile distribution for each column
summary_stats = df_final.describe(percentiles=[0.25, 0.5, 0.75]).transpose()

# Combine all the results into a single DataFrame
result_df = pd.DataFrame({
    'Count':counts,
    'Distinct Values': distinct_counts,
    'Null Count': null_counts,
    'Mean': summary_stats['mean'],
    'Std': summary_stats['std'],
    'Minimum Value': summary_stats['min'],
    'Percentile Distribution (25%)': summary_stats['25%'],
    'Percentile Distribution (50%)': summary_stats['50%'],
    'Percentile Distribution (75%)': summary_stats['75%'],
    'Maximum Value': summary_stats['max']
})



result_df

,Count,Distinct Values,Null Count,Mean,Std,Minimum Value,Percentile Distribution (25%),Percentile Distribution (50%),Percentile Distribution (75%),Maximum Value
CrewSize,2177,59,341,21.719339,9.548722,2.000001,14.000001,18.000000,32.000000,47.0000
EnergyCostPerUnit,2177,589,341,0.133709,0.274513,0.000137,0.037867,0.078646,0.154080,7.1904
FactCode,2518,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LineNum,2518,25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MOQ,0,0,2518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ThruputPerHour,2518,358,0,126.620096,88.463435,2.000000,92.600000,105.000000,157.142857,1800.0000
Unified code,2518,550,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
retain_flag,2518,2,0,0.864575,0.342245,0.000000,1.000000,1.000000,1.000000,1.0000


# FGSKU Count

In [21]:
df_month_demand_ret_1 = df_demand[df_demand['Ret_Flag'] == 1]
df_month_demand_ret_1=df_month_demand_ret_1['FGSKUCode'].nunique()

df_month_demand_merge_ret_1 = df_merge[df_merge['Ret_Flag'] == 1]
df_month_demand_merge_ret_1=df_month_demand_merge_ret_1['FGSKUCode'].nunique()


# VOLUME(CARTONS)

In [22]:

df_demand_carton=df_demand[df_demand['UM'] == 'CARTON']
df_demand_carton_merge =df_merge[df_merge['UM'] == 'CARTON']

df_demand_carton_ret_1 = df_demand_carton[df_demand_carton['Ret_Flag'] == 1]
df_demand_carton_ret_1=df_demand_carton_ret_1['Demand'].sum().sum()

df_demand_carton_merge_ret_1 = df_demand_carton_merge[df_demand_carton_merge['Ret_Flag'] == 1]
df_demand_carton_merge_ret_1 = df_demand_carton_merge_ret_1['Demand'].sum().sum()


# VOLUME(KG)

In [23]:

df_demand_kg=df_demand[df_demand['UM'] == 'KILO GRAM']
df_demand_merge_kg=df_merge[df_merge['UM'] == 'KILO GRAM']

#main
df_demand_kg_ret_1 = df_demand_kg[df_demand_kg['Ret_Flag'] == 1]
df_demand_kg_ret_1=df_demand_kg_ret_1['Demand'].sum().sum()

df_demand_merge_kg_ret_1 = df_demand_merge_kg[df_demand_merge_kg['Ret_Flag'] == 1]
df_demand_merge_kg_ret_1=df_demand_merge_kg_ret_1['Demand'].sum().sum()

In [24]:
#final table
#Storing result in a single df
result_df_1 = pd.DataFrame({
    'Demand Output (Ret_Flag=1)': [df_month_demand_ret_1,df_demand_carton_ret_1,df_demand_kg_ret_1]
   , 'Prod output': [df_month_demand_merge_ret_1,df_demand_carton_merge_ret_1,df_demand_merge_kg_ret_1]
}, index=['FGSKU','Volume (cartons)','Volume (kgs)'])

# Display the result 
print(result_df_1.to_string(float_format='{:,.2f}'.format))

                  Demand Output (Ret_Flag=1)   Prod output
FGSKU                                 551.00        550.00
Volume (cartons)               11,077,837.49 11,077,717.49
Volume (kgs)                      158,211.31    158,211.31


In [25]:
dfs = [df_final, result_df, result_df_1]

# Create a new Excel workbook

wb = Workbook()
 
# Remove default sheet created by openpyxl

default_sheet = wb['Sheet']

wb.remove(default_sheet)

 
# Create a named style for bold headings

bold_style = NamedStyle(name='bold_style')

bold_style.font = Font(bold=True)  # Set font color to white
 
# Define the desired sheet names

sheet_names = ['Production_output', 'QC_output']
 
# Define blue fill

blue_fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")
 
# Loop through each DataFrame and add it to a new sheet in the workbook

for df, sheet_name in zip(dfs, sheet_names):

    # Create a new sheet

    ws = wb.create_sheet(title=sheet_name)
 
    # Write DataFrame to the sheet, starting from cell A1
   # Write header row

    if sheet_name == 'Production_output':

        header_row = [str(col) for col in df.columns]

        ws.append(header_row)

        for idx, row in df.iterrows():

            ws.append(row.tolist())

        for cell in ws[1]:

            cell.style = bold_style

            cell.fill = blue_fill

        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, max_col=ws.max_column):

            for cell in row:

                cell.border = Border(left=Side(style='thin'),

                                     right=Side(style='thin'),

                                     top=Side(style='thin'),

                                     bottom=Side(style='thin'))

        ws.sheet_view.showGridLines = False

        for col in ws.columns:

            max_length = 0

            for cell in col:

                try:

                    if len(str(cell.value)) > max_length:

                        max_length = len(cell.value)

                except:

                    pass

            adjusted_width = (max_length + 2) * 1.2

            ws.column_dimensions[col[0].column_letter].width = adjusted_width
            # Add one row and column at the top
        ws.insert_rows(idx=1)
        ws.insert_cols(idx=1)
        ws.column_dimensions['A'].width = 3
 

    elif sheet_name == 'QC_output':

        df = result_df

        header_row = ['Column Name'] + [str(col) for col in df.columns]

        ws.append(header_row)

        for idx, row in df.iterrows():

            ws.append([idx]+row.tolist())

        for cell in ws[1]:

            cell.style = bold_style

            cell.fill = blue_fill

        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, max_col=ws.max_column):

            for cell in row:

                cell.border = Border(left=Side(style='thin'),

                                     right=Side(style='thin'),

                                     top=Side(style='thin'),

                                     bottom=Side(style='thin'))

        for col in ws.columns:

            max_length = 0

            for cell in col:

                try:

                    if len(str(cell.value)) > max_length:

                        max_length = len(cell.value)

                except:

                    pass

            adjusted_width = (max_length + 2) * 1.2

            ws.column_dimensions[col[0].column_letter].width = adjusted_width
 
        for _ in range(4):

            ws.append([])

        df = result_df_1

        header_row = ['Data'] + [str(col) for col in df.columns]

        ws.append(header_row)

        for idx, row in df.iterrows():

            ws.append([idx]+row.tolist())

        for idx, cell in enumerate(ws[14], start=1):

                if idx <= 3:

                    cell.style = bold_style

                    cell.fill = blue_fill

        for row in ws.iter_rows(min_row=14, max_row=17, max_col=3):

            for cell in row:

                cell.border = Border(left=Side(style='thin'),

                                     right=Side(style='thin'),

                                     top=Side(style='thin'),

                                     bottom=Side(style='thin'))

        ws.sheet_view.showGridLines = False

        for col in ws.columns:

            max_length = 0

            for cell in col:

                try:

                    if len(str(cell.value)) > max_length:

                        max_length = len(cell.value)

                except:

                    pass

            adjusted_width = (max_length + 2) * 1.2

            ws.column_dimensions[col[0].column_letter].width = adjusted_width
# Add one row and column at the top
ws.insert_rows(idx=1)
ws.insert_cols(idx=1)
ws.column_dimensions['A'].width = 3
# Save the workbook

wb.save('03-05-2024_Prod_Automation.xlsx')
print("Completed Successfully")

Completed Successfully
